In [1]:
#largest in representation
import numpy as np

def largest(b,t,l,u):
    y = np.float64(0)
    b = np.float64(b)
    t = int(t)
    l = np.float64(l) 
    u = np.float64(u)     
    for i in range(t):
        y = y + b**(-i-1)
        
    return b**u*y

In [3]:
def lower_triangular_solve(A, b):
    """
    Solve the system  A x = b  where A is assumed to be lower triangular,
    i.e. A(i,j) = 0 for j > i, and the diagonal is assumed to be nonzero,
    i.e. A(i,i) != 0.
    
    The code checks that A is lower triangular and converts A and b to
    double precision before computing.

    ARGUMENTS:  A   lower triangular n x n array
                b   right hand side column n-vector

    RETURNS:    x   column n-vector solution
    """

    # we should take care to ensure that arrays are stored with the correct type - float!
    A = A.astype(np.float64)
    b = b.astype(np.float64)
     
    # check sizes of A and b match appropriately
    nb=len(b)
    n, m = A.shape
    if n != m:
        raise ValueError(f'A is not a square matrix! {A.shape=}')
    if n != nb:
        raise ValueError(f'shapes of A and b do not match! {A.shape=} {b.shape=}')
    
    # checks whether A is lower triangular
    for i in range(n):
        for j in range(i+1,n):
            if not np.isclose(A[i, j], 0.0):
                raise ValueError(f'A is not lower triangular! {A[i, j]=}')

    # checks whether A has zero diagonal element
    for i in range(n):
        if np.isclose(A[i, i], 0.0):
            raise ValueError(f'A[{i}, {i}] is zero')
    
    # create a new array to store the results
    x = np.empty_like(b)
    
    # perform forward substitution
    x[0] = b[0] / A[0, 0]
    for i in range(1,n):
        x[i] = b[i] / A[i, i]
        for j in range(i):
            x[i] = x[i] - A[i,j]*x[j]/A[i, i]
        
    return x

In [4]:
def upper_triangular_solve(A, b):
    """
    Solve the system  A x = b  where A is assumed to be lower triangular,
    i.e. A(i,j) = 0 for j > i, and the diagonal is assumed to be nonzero,
    i.e. A(i,i) != 0.
    
    The code checks that A is lower triangular and converts A and b to
    double precision before computing.

    ARGUMENTS:  A   lower triangular n x n array
                b   right hand side column n-vector

    RETURNS:    x   column n-vector solution
    """

    # we should take care to ensure that arrays are stored with the correct type - float!
    A = A.astype(np.float64)
    b = b.astype(np.float64)
      
    # check sizes of A and b match appropriately
    nb=len(b)
    n, m = A.shape
    if n != m:
        raise ValueError(f'A is not a square matrix! {A.shape=}')
    if n != nb:
        raise ValueError(f'shapes of A and b do not match! {A.shape=} {b.shape=}')
    
    # check A is upper triangular
    for i in range(n):
        for j in range(0,i):
            if not np.isclose(A[i, j], 0.0):
                raise ValueError(f'A is not upper triangular! {A[i, j]=}')

    # checks whether A has zero diagonal element
    for i in range(n):
        if np.isclose(A[i, i], 0.0):
            raise ValueError(f'A[{i}, {i}] is zero')
    
    #create a new array to store the results
    x = np.empty_like(b)
    
    # perform backwards substitution
    x[n-1] = b[n-1] / A[n-1, n-1]
    for i in range(2,n+1):
        x[n-i] = b[n-i] / A[n-i, n-i]
        for j in range(n-i+1, n):
            x[n-i] = x[n-i] - A[n-i,j]*x[j] / A[n-i, n-i]
        
    return x

In [5]:
def gaussian_elimination(A, b, verbose=False):
    # To ensure that arrays are stored in double precision.
    A = A.astype(np.float64)
    b = b.astype(np.float64)
     
    # size of solution vector / the square matrix A
    n=len(b) # or   n, n = A.shape
        
    # check sizes of A and b match appropriately
    nb=len(b)
    n, m = A.shape
    if n != m:
        raise ValueError(f'A is not a square matrix! {A.shape=}')
    if n != nb:
        raise ValueError(f'shapes of A and b do not match! {A.shape=} {b.shape=}')
    
    if verbose:
        print('starting system\n', A, b)

    # perform forward elimination
    for i in range(n):  
        # eliminate column i
        if verbose:
            print(f'eliminating column {i}')
        
        # check diagonal
        if np.isclose(A[i, i], 0.0):
            raise ValueError(f'A has zero on diagonal! A[{i}, {i}] = 0')

        # row j <- row j - (a_{ji} / a_{ii}) row i
        for j in range(i+1, n):
            if verbose:
                print(f'row {j} <- row {j} - {A[j, i] / A[i, i]} row {i}')
            factor = A[j, i] / A[i, i]
            for k in range(0, n):
                A[j, k] = A[j, k] - factor * A[i, k]
            b[j] = b[j] - factor * b[i]
        
        if verbose:
            print('new system\n', A, b)

    return upper_triangular_solve(A, b)   

U = np.array([[2, 1, 4], [0, 1.5, 0], [0, 0, 2]], dtype=np.double)
b = np.array([[12], [3], [4]], dtype=np.double)

#print(gaussian_elimination(U, b, verbose=True))

In [6]:
def Jacobi_iteration(A, b, max_iteration, x0 = None):
    # we should take care to ensure that arrays are stored with the correct type - float!
    A = A.astype(np.float64)
    b = b.astype(np.float64)
     
    # check sizes of A and b match appropriately
    nb=len(b)
    n, m = A.shape
    if n != m:
        raise ValueError(f'A is not a square matrix! {A.shape=}')
    if n != nb:
        raise ValueError(f'shapes of A and b do not match! {A.shape=} {b.shape=}')

    # check diagonal is non zero
    for i in range(n):
        if np.isclose(A[i, i], 0):
            raise ValueError(f'A[{i}, {i}] is zero')

    # construct iteration matrices
    P=np.zeros([n,n])    # matrix P = D^{-1}(L+U)
    p=np.zeros(n)        # vector p = D^{-1} b
    for i in range(n):
        p[i]=b[i]/A[i,i] 
        for j in range(n):
             P[i,j] = A[i,j]/A[i,i]
        P[i,i] = 0
        
    #create a new array to store the results, initialised as zero
    if x0 is None:
        x = np.zeros_like(b)
    else:
        x = x0.copy()
    
    # perform iteration x <- p - P * x
    for it in range(max_iteration):
        xnew = np.empty_like(x)
        for i in range(n):
            xnew[i] = p[i]
            for j in range(n):
                xnew[i] -= P[i, j] * x[j]
        x = xnew.copy()
                
    return x

In [7]:
def Gauss_Seidel_iteration(A, b, max_iteration, x0 = None):
    # we should take care to ensure that arrays are stored with the correct type - float!
    A = A.astype(np.float64)
    b = b.astype(np.float64)
     
    # check sizes of A and b match appropriately
    nb=len(b)
    n, m = A.shape
    if n != m:
        raise ValueError(f'A is not a square matrix! {A.shape=}')
    if n != nb:
        raise ValueError(f'shapes of A and b do not match! {A.shape=} {b.shape=}')

    for i in range(n):
        if np.isclose(A[i, i], 0):
            raise ValueError(f'A[{i}, {i}] is zero')

    # do not construct iteration matrices explicitly
    LD = np.zeros_like(A)
    U = np.zeros_like(A)
    for i in range(n):
        for j in range(n):
            if i < j:
                U[i, j] = A[i, j]
            else:
                LD[i, j] = A[i, j]
    
    # p = (L + D)^{-1} b --> found by solving triangular system
    # (L + D) p = b
    p = lower_triangular_solve(LD, b)
      
    #create a new array to store the results, initialised as zero
    if x0 is None:
        x = np.zeros_like(b)
    else:
        x = x0.copy()
        
    # perform iteration x <- p - P * x
    # (L+D)(xnew - p) = U*x
    Ux = np.empty_like(x)
    for it in range(max_iteration):
        for i in range(n):
            Ux[i] = 0.0
            for j in range(i+1, n):
                Ux[i] += U[i, j] * x[j]
        Px = lower_triangular_solve(LD, Ux)
        x = p - Px
                
    return x

# Test different linear solvers starting from the above two-dimensional linear system
A = np.array([[2, -1, 0], [-1, 1, 3], [1,0,1]])
b = np.array([1, 3, 2])
x_exact = np.array([0,0,0])

# numpy linear solver
x0 = np.linalg.solve(A,b)
print("Solution by numpy solver:", x0)

x = Jacobi_iteration(A, b, 26, x_exact)
print("Solution by Jacobi iteration: ",x)
#print("Error: ", x - x_exact)
print("Residual: ", np.matmul(A,x)-b)

x = Gauss_Seidel_iteration(A, b, 26, x_exact)
print("Solution by Gauss Seidel iteration: ",x)
#print("Error: ", x - x_exact)
print("Residual: ", np.matmul(A,x)-b)

Solution by numpy solver: [1. 1. 1.]
Solution by Jacobi iteration:  [ -508 -1293   386]
Residual:  [ 276  370 -124]
Solution by Gauss Seidel iteration:  [ 5122. 15362. -5120.]
Residual:  [-5119. -5123.     0.]


In [8]:
def Gaussian_elimination_pivoting(A, b, verbose=False):
    # To ensure that arrays are stored in double precision.
    A = A.astype(np.float64)
    b = b.astype(np.float64)
     
    # size of solution vector / the square matrix A
    n=len(b) # or   n, n = A.shape
        
    # check sizes of A and b match appropriately
    nb=len(b)
    n, m = A.shape
    if n != m:
        raise ValueError(f'A is not a square matrix! {A.shape=}')
    if n != nb:
        raise ValueError(f'shapes of A and b do not match! {A.shape=} {b.shape=}')
    
    if verbose:
        print('starting system\n', A, b)
  
    # perform forward elimination
    for i in range(n):          
        # find the index of the maximal value in column i on or below
        # the diagonal of A
        maximum = abs(A[i,i])
        max_index = i
        for j in range(i+1,n):
            if abs(A[j,i]) > maximum :
                maximum = abs(A[j,i])               
                max_index = j   
                                       
        
        # swap two max_indexs: i and max_index[i]
        temp = b[i]
        b[i] = b[max_index]
        b[max_index] = temp
        for j in range(n):
            temp = A[i,j]
            A[i,j] = A[max_index,j]
            A[max_index,j] = temp  
            
        
        # check diagonal
        if np.isclose(A[i, i], 0.0):
            raise ValueError(f'A has zero on diagonal! A[{i}, {i}] = 0') 

        # row j <- row j - (a_{ji} / a_{ii}) row i
        for j in range(i+1, n):
            if verbose:
                print(f'row {j} <- row {j} - {A[j, i] / A[i, i]} row {i}')
            factor = A[j, i] / A[i, i]
            for k in range(0, n):
                A[j, k] = A[j, k] - factor * A[i, k]
            b[j] = b[j] - factor * b[i]
        
    return upper_triangular_solve(A, b)

A = np.array([[-10, 2, 0, 67], [-2, 50, -77, 1.e-5], [1, 7, 30, 8], [-10, -7, 0.001, 80]])
b = np.array([1, 2, 9, 0])

# numpy linear solvers
x0 = np.linalg.solve(A,b)
#x0 = np.linalg.inv(A).dot(b)
print("Solution by numpy solver:", x0)

x = Gaussian_elimination_pivoting(A, b)
print("Gaussian elimination: ",x)
print("Residual: ", np.matmul(A,x)-b)

# extra test to ensure we have solved the problem
np.testing.assert_almost_equal(np.linalg.norm(b - A @ x), 0.0)

Solution by numpy solver: [0.9244595  0.31826746 0.15668124 0.14340388]
Gaussian elimination:  [0.9244595  0.31826746 0.15668124 0.14340388]
Residual:  [ 0.00000000e+00 -1.77635684e-15 -1.77635684e-15  0.00000000e+00]


In [9]:
A_real = np.array([3, 1.5, 2,4 ,2.2, 2, 6, 4, -9], dtype=np.float64)
I_row = np.array([0, 0, 1, 1, 1, 2, 2, 3, 3], dtype=np.int32)
I_col = np.array([0, 2, 0, 1, 2, 1, 2, 2, 3], dtype=np.int32) 

nonzero = len(A_real)
dim = 4
y = np.zeros(dim) + 1.
z = np.zeros(dim)

for k in range(nonzero):
    z[I_row[k]] = z[I_row[k]] + A_real[k] * y[I_col[k]]

#print(z)

In [10]:
 def Euler_method(t0, d0, dt, n):
    d = np.zeros(n+1)
    d[0] = d0

    t = np.zeros(n+1)
    for i in range(n+1):
        t[i] = t0 + i*dt
    
    for i in range(1, n+1):          
        d[i] = d[i-1] + dt * f(t[i-1], d[i-1])
                                       
    return t, d

In [11]:
 def midpoint_method(t0, d0, dt, n):
    d = np.zeros(n+1)
    d[0] = d0

    t = np.zeros(n+1)
    for i in range(n+1):
        t[i] = t0 + i*dt
    
    for i in range(1, n+1):
        d_half = d[i-1] + 0.5 * dt * f(t[i-1], d[i-1])
        d[i] = d[i-1] + dt * f((t[i-1]+t[i])/2.0, d_half)
                                       
    return t, d

In [12]:
def newton(f, df, x0, tol):
    x = x0
    y = f(x)
    it = 0
    while abs(y) > tol:   # iterate until less than or eq tol
        x = x - y / df(x)  # apply one Newton iteration
        y = f(x)           # reevaluate f at new estimate
        it = it + 1

    return x, it

def bisection(f, x0, x1, tol):
    it = 0
    x = (x0 + x1)/2.0
    while abs(f(x)) > tol:
        it = it +1
        x = (x0 + x1)/2.0
        if abs(x) < 1.e-6: return x
        if f(x)*f(x0) < 0:
            x1 = x
        else:
            x0 = x       

    return x, it

def f(t):
    return t*t*t - 2.0

def df(t):
    return 3.*t*t


x, it = newton(f, df, 2.0, 1.e-6)
print(f"Newton method: {x} after {it} iterations")
np.testing.assert_allclose(abs(f(x)), 0.0, atol=1.0e-6)

x, it = bisection(f, 0.0, 2.0, 1.e-6)
print(f"Bisection method: {x} after {it} iterations")
np.testing.assert_allclose(abs(f(x)), 0.0, atol=1.0e-6)

Newton method: 1.2599210498953948 after 5 iterations
Bisection method: 1.2599210739135742 after 21 iterations


In [13]:
def secant(f, x0, x1, tol):
    x = x1
    it = 0
    while abs(f(x)) > tol:   # iterate until less than or eq tol
        x = x - f(x1) *(x1-x0) / (f(x1) - f(x0))  # apply one Newton iteration
        x0 = x1
        x1 = x
        it = it + 1

    return x, it

def f(x):
    return x*x*x - 6.0*x*x +9.0*x

x, it = secant(f, 4.0, 5.0, 1.e-6)
print(f"The secant method: {x} after {it} iterations")
np.testing.assert_allclose(abs(f(x)), 0.0, atol=1.0e-6)

The secant method: 3.0004913634209194 after 17 iterations


In [14]:
def Residual(A,x,b):
    return np.matmul(A,x)-b

def jacobi(A,b,N=25,x=None):
    """Solves the equation Ax=b via the Jacobi iterative method."""
    # Create an initial guess if needed  
    # To ensure that arrays are stored in double precision.
    #A = A.astype(np.float64)
    #b = b.astype(np.float64)
    if x is None:
        x = np.zeros(len(A[0]))

    # Create a vector of the diagonal elements of A                                                                                                                                                
    # and subtract them from A                                                                                                                                                                     
    D = np.diag(A)
    R = A - np.diagflat(D)

    # Iterate for N times                                                                                                                                                                          
    for i in range(N):
        x = (b - np.dot(R,x)) / D
        print(f"{i} -> x = {x} | r Mag = {Residual(A,b,x)[0]:.5}")
    return x

A = np.array([[2.,-1.,0.],[-1.,1.,3.],[1.,0.,1.]])
b = np.array([1.,3.,2.])
guess = np.array([0.,0.,0.])

sol = jacobi(A,b,N=2,x=guess)
print(sol)

0 -> x = [0.5 3.  2. ] | r Mag = -1.5
1 -> x = [ 2.  -2.5  1.5] | r Mag = -3.0
[ 2.  -2.5  1.5]


In [15]:
def Residual(A,x,b):
    return b-np.matmul(A,x)

def gaussSeidel(a,b,x,its):
    
    n = x.shape[0]
    
    for it in range(its):
        print(f"{it} -> x = {x} | r Mag = {Residual(a,b,x)[0]:.5}")
        
        
        for i in range(n):
            sum = 0
            
            for j in range(n):
                sum += a[i][j] *x[j]
                
            x[i] += (1/a[i][i])*(b[i] - sum)
            
    return x,Residual(a,b,x)
    
    
A = np.array([[2.,-1.,0.],[-1.,1.,3.],[1.,0.,1.]])
b = np.array([1.,3.,2.])
guess = np.array([0.,0.,0.])

result,res = gaussSeidel(A,b,guess,25)
print(result, res[0])

0 -> x = [0. 0. 0.] | r Mag = 1.0
1 -> x = [0.5 3.5 1.5] | r Mag = 1.5
2 -> x = [ 2.25  0.75 -0.25] | r Mag = 3.25
3 -> x = [0.875 4.625 1.125] | r Mag = 1.875
4 -> x = [ 2.8125  2.4375 -0.8125] | r Mag = 3.8125
5 -> x = [1.71875 7.15625 0.28125] | r Mag = 2.7188
6 -> x = [ 4.078125  6.234375 -2.078125] | r Mag = 5.0781
7 -> x = [ 3.6171875 12.8515625 -1.6171875] | r Mag = 4.6172
8 -> x = [ 6.92578125 14.77734375 -4.92578125] | r Mag = 7.9258
9 -> x = [ 7.88867188 25.66601562 -5.88867188] | r Mag = 8.8887
10 -> x = [ 13.33300781  33.99902344 -11.33300781] | r Mag = 14.333
11 -> x = [ 17.49951172  54.49853516 -15.49951172] | r Mag = 18.5
12 -> x = [ 27.74926758  77.24780273 -25.74926758] | r Mag = 28.749
13 -> x = [ 39.12390137 119.3717041  -37.12390137] | r Mag = 40.124
14 -> x = [ 60.18585205 174.55755615 -58.18585205] | r Mag = 61.186
15 -> x = [ 87.77877808 265.33633423 -85.77877808] | r Mag = 88.779
16 -> x = [ 133.16816711  393.50450134 -131.16816711] | r Mag = 134.17
17 -> x = [ 